In [1]:
import pandas as pd
import numpy as np
import infrastructure
import filepath

# 1. Import Data

In [2]:
#Use Infrastructure.py 
mamba = infrastructure.get_mamba()
lexi_sms = pd.read_csv(filepath.output_folder + 'SS_LC_merged_data.csv')
lexi_sms = infrastructure.transform_sms_df(lexi_sms)
lexi_sms['Hitpath Offer ID'] = lexi_sms['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
#import offer list from mamba  
offerlist = infrastructure.get_ss_offer()
# import oms 
oms = infrastructure.get_smartsheet('offers_sms')


/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Jumppage_Testing/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_9459/2619234961.py:3: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  lexi_sms = pd.read_csv(filepath.output_folder + 'SS_LC_merged_data.csv')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Jumppage_Testing/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_dateti

# 2. Find Jump Page that awaiting to be tested

In [3]:
# find jump page offer that haven't been tested  
offerlist_jumppage = offerlist[offerlist['SS Offers (updated)'].str.contains('v',na = False)]['SS Offers (updated)'].str.split(expand = True)[0]
oms['Hitpath Offer ID'] = oms['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
lexi_sms['Jump Page Offer'] = lexi_sms['Offer'].str.split(expand = True)[0] 
# Find jump page version in the lexi_sms and get the stat from lexi 
lexi_sms_jumppage = lexi_sms[( lexi_sms['Jump Page Version']!=0) &( lexi_sms['Jump Page Version'].isna()==False) ]
lexi_sms_jumppage['Jump Page Offer'] = lexi_sms_jumppage['Offer'].str.split(expand = True)[0]
jp_stat = lexi_sms_jumppage.groupby(['Jump Page Offer','shortcode_DP.SV','Date']).agg({"Hitpath Offer ID":'count', 'Revenue': 'sum','Jump Page Clicks':'sum','Delivered':'sum'}).reset_index()
jp_stat1 = jp_stat.groupby(['Jump Page Offer']).agg({"Hitpath Offer ID":'count','Date':'max', 'Revenue': 'sum','Jump Page Clicks':'sum','Delivered':'sum'}).reset_index()
jp_stat1['Jump Page CTR'] = jp_stat1['Jump Page Clicks']/jp_stat1['Delivered']
jp_stat1['eCPM'] = jp_stat1['Revenue'] * 1000/jp_stat1['Delivered']
jp_stat1.rename(columns = {"Hitpath Offer ID":'# Drops','Date':'Last Date'},inplace = True)  

# Find the jump page offer that scheduled in the upcoming schedule 
mamba_today = mamba[mamba['Date'] >= min(pd.to_datetime('today'), lexi_sms['Date'].max())] 
mamba_today_jp = mamba_today[mamba_today['Offer'].str.contains('|'.join(offerlist_jumppage.unique().tolist()),na = False)]
if len(mamba_today_jp) > 0:
    mamba_today_jp['Jump Page Offer'] = mamba_today_jp['Offer'].str.split(expand = True)[0]
    # group by jump page offer and get # drops 
    mamba_today_jp['# Drops'] = mamba_today_jp.groupby('Jump Page Offer')['Offer'].transform('count')
    mamba_today_jp['Last Date'] = mamba_today_jp.groupby('Jump Page Offer')['Date'].transform('max')
    mamba_today_jp = mamba_today_jp[['Jump Page Offer','# Drops','Last Date']] 

    # concat the stat from lexi and the upcoming schedule
    all_jump_page_schedule = pd.concat([jp_stat1,mamba_today_jp],axis = 0)
    all_jump_page_schedule = all_jump_page_schedule.groupby('Jump Page Offer').agg({'# Drops':'sum','Last Date':'max','Revenue':'sum','Jump Page Clicks':'sum','Delivered':'sum'}).reset_index()
else: 
    all_jump_page_schedule = jp_stat1.copy()
all_jump_page_schedule = all_jump_page_schedule
all_jump_page_schedule['Jump Page CTR'] = all_jump_page_schedule['Jump Page Clicks']/all_jump_page_schedule['Delivered']
all_jump_page_schedule['eCPM'] = all_jump_page_schedule['Revenue'] * 1000/all_jump_page_schedule['Delivered']
all_jump_page_schedule['Hitpath Offer ID'] = all_jump_page_schedule['Jump Page Offer'].str.split("v",expand = True)[0]
all_jump_page_schedule = all_jump_page_schedule.merge(oms[['Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status']],on = 'Hitpath Offer ID',how = 'left')
# Find the jump page offer that haven't been tested
# create a dataframe that has offerlist_jumppage  as first column, and the second column is 2 
untested_jp = pd.DataFrame(offerlist_jumppage.unique())
untested_jp['count'] = 3
untested_jp.rename(columns = {0:'Jump Page Offer'},inplace = True)
# merge with all_jump_page_schedule 
untested_jp = untested_jp.merge(all_jump_page_schedule[['Jump Page Offer','# Drops']],on = 'Jump Page Offer',how = 'left')
untested_jp['Test count'] = untested_jp['count'] - untested_jp['# Drops']
untested_jp = untested_jp[(untested_jp['Test count'] > 0) | (untested_jp['# Drops'].isna())]
# remove column 
untested_jp = untested_jp.drop(columns = ['count','# Drops'])
untested_jp = untested_jp.fillna(3)
untested_jp['Hitpath Offer ID'] = untested_jp['Jump Page Offer'].str.split("v",expand = True)[0]

untested_jp_offer =  offerlist_jumppage[~offerlist_jumppage.isin(all_jump_page_schedule['Jump Page Offer'])].str.split("v",expand = True)[0].unique().tolist()
tested_30d = all_jump_page_schedule[all_jump_page_schedule['Last Date']>= pd.to_datetime('today') - pd.DateOffset(days = 30)]['Jump Page Offer'].str.split("v",expand = True)[0].unique().tolist()
# Find stat for the untested jump page offer
# last 60 days data from lexi 
lexi_sms_last60 = lexi_sms[lexi_sms['Date'] >= pd.to_datetime('today') - pd.DateOffset(days = 60)]
# send strategy = p 
lexi_sms_last60_p = lexi_sms_last60[lexi_sms_last60['Send Strategy'] == 'P']
lexi_sms_last60_p.loc[lexi_sms_last60_p['Code_Type'] == 'Toll Free', 'Jump Page Clicks'] = np.nan
# group by Hitpath_Offer_ID and get the stat
offer_performance = lexi_sms_last60_p.groupby("Hitpath Offer ID").agg({"Date":'count', 'Revenue': 'sum','Jump Page Clicks':'sum','Delivered':'sum','Clicks':'sum'}).reset_index()
offer_performance.rename(columns = {'Date':'# Drops'},inplace = True)

offer_performance['Text CTR'] = offer_performance['Clicks']/offer_performance['Delivered'] 
offer_performance['eCPM'] = offer_performance['Revenue'] * 1000/offer_performance['Delivered']
offer_performance['Jump Page CTR'] = offer_performance['Jump Page Clicks']/offer_performance['Delivered']
offer_performance['Jump Page Click Rate'] = offer_performance['Jump Page Clicks']/offer_performance['Clicks']
offer_performance['JPCTR Normalized']=(offer_performance['Jump Page CTR']-offer_performance['Jump Page CTR'].min())/(offer_performance['Jump Page CTR'].max()-offer_performance['Jump Page CTR'].min())
offer_performance['eCPM Normalized']=(offer_performance['eCPM']-offer_performance['eCPM'].min())/(offer_performance['eCPM'].max()-offer_performance['eCPM'].min())
offer_performance['JPCTR50'] = offer_performance['JPCTR Normalized'] + offer_performance['eCPM Normalized']
# we want to find the offer that has the lower jump page click rate and the high text CTR
offer_performance['combine_index'] = offer_performance['Text CTR'].rank(ascending = False) + offer_performance['Jump Page Click Rate'].rank(ascending = True)

# filter out the live offer
good_offer_offer_performance = offer_performance[offer_performance['eCPM'] > 7]
offer_performance = offer_performance.merge(oms[['Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status']],on = 'Hitpath Offer ID',how = 'left')
offer_performance = offer_performance[offer_performance['Status'].str.contains('Live', na = False)]
# ecpm must above $7 
poor_offer = offer_performance[offer_performance['eCPM'] <= 7]


offer_performance.sort_values(by = 'combine_index',ascending = True,inplace = True)
## Monday Task: 
# find hitpath offer ID that we need to requested more jump page based on the perfromance, make sure they are live offer/ live offer wall 
offer_performance_awaiting_test = good_offer_offer_performance.merge(untested_jp, how  = 'right', on = 'Hitpath Offer ID')
offer_performance_awaiting_test = offer_performance_awaiting_test.merge(oms[['Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status']],on = 'Hitpath Offer ID',how = 'left')
offer_performance_awaiting_test = offer_performance_awaiting_test[offer_performance_awaiting_test['Status'].str.contains('Live', na = True)].sort_values(by = 'combine_index',ascending = True)
offer_performance_awaiting_test1 = offer_performance_awaiting_test[['Hitpath Offer ID','Jump Page Offer','Test count']]
# find jump page offer that haven't been tested  or recently been tested by the last 30 days and sort that by the value 
offer_performance_requesting_jp = offer_performance[(offer_performance['Hitpath Offer ID'].isin(untested_jp_offer+tested_30d) == False) ]
# we will request jump page for offer that the jump page click rate is lower than average 
request_jp =offer_performance_requesting_jp[ offer_performance_requesting_jp['Jump Page Click Rate']<= offer_performance_requesting_jp['Jump Page Click Rate'].mean()]


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_9459/2871121908.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lexi_sms_jumppage['Jump Page Offer'] = lexi_sms_jumppage['Offer'].str.split(expand = True)[0]


In [4]:
# add a column to count the number of jump page version by the same 'Hitpath Offer ID','shortcode_DP.SV','Date' the group
lexi_sms = lexi_sms.loc[lexi_sms['Delivered']!=0]
lexi_sms['JPV count'] = lexi_sms.groupby(['Hitpath Offer ID','shortcode_DP.SV','Date'])['Jump Page Version'].transform('nunique')
lexi_sms_JPT = lexi_sms[lexi_sms['JPV count'] > 1].groupby(['Hitpath Offer ID','shortcode_DP.SV','Date','Jump Page Offer','Jump Page Version','Offer', 'SC_DP&Pub']).agg({"Revenue":'sum','Delivered':'sum','Clicks':'sum','Jump Page Clicks':'sum'}).reset_index()
lexi_sms_JPT['eCPM'] = lexi_sms_JPT['Revenue'] * 1000/lexi_sms_JPT['Delivered']
lexi_sms_JPT['Jump Page CTR'] = lexi_sms_JPT['Jump Page Clicks']/offer_performance['Delivered']
lexi_sms_JPT['Jump Page Click Rate'] = lexi_sms_JPT['Jump Page Clicks']/lexi_sms_JPT['Clicks']
lexi_sms_JPT['JPCTR Normalized']=(lexi_sms_JPT['Jump Page Click Rate']-lexi_sms_JPT['Jump Page Click Rate'].min())/(lexi_sms_JPT['Jump Page Click Rate'].max()-lexi_sms_JPT['Jump Page Click Rate'].min())
lexi_sms_JPT['eCPM Normalized']=(lexi_sms_JPT['eCPM']-lexi_sms_JPT['eCPM'].min())/(lexi_sms_JPT['eCPM'].max()-lexi_sms_JPT['eCPM'].min())
lexi_sms_JPT['JPCTR50'] = lexi_sms_JPT['JPCTR Normalized'] + lexi_sms_JPT['eCPM Normalized']
lexi_sms_JPT_simplified =lexi_sms_JPT[['Date','Offer','Hitpath Offer ID','Jump Page Offer','Jump Page Version','SC_DP&Pub','Revenue','Delivered','Jump Page Clicks','eCPM','Jump Page Click Rate','JPCTR50']]
lexi_sms_JPT_simplified = lexi_sms_JPT_simplified.merge(oms[['Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status']],on = 'Hitpath Offer ID',how = 'left')

# get a conclusion column by comparing the jump page click rate and eCPM  by the same 'Hitpath Offer ID','shortcode_DP.SV','Date' the group
#lexi_sms_JPT_simplified['jp_rank']= lexi_sms_JPT_simplified.groupby(['Date','Hitpath Offer ID','SC_DP&Pub'])['Jump Page Click Rate'].rank(ascending = False,method='dense')
#lexi_sms_JPT_simplified['ecpm_rank'] = lexi_sms_JPT_simplified.groupby(['Date','Hitpath Offer ID','SC_DP&Pub'])['eCPM'].rank(ascending = False,method='dense')
#lexi_sms_JPT_simplified['conclusion'] = lexi_sms_JPT_simplified['jp_rank'] + lexi_sms_JPT_simplified['ecpm_rank']
lexi_sms_JPT_simplified['Success Test'] = lexi_sms_JPT_simplified['JPCTR50'] == lexi_sms_JPT_simplified.groupby(['Date','Hitpath Offer ID','SC_DP&Pub'])['JPCTR50'].transform('max')

lexi_jpt_summary = lexi_sms_JPT_simplified.groupby(['Hitpath Offer ID','Jump Page Version','Jump Page Offer']).agg({'Success Test':'sum','Revenue':'count','Jump Page Clicks': 'sum','Delivered':'sum'}).reset_index()
lexi_jpt_summary.rename(columns = {'Revenue':'Total Test'},inplace = True)
lexi_jpt_summary['Success Rate'] = lexi_jpt_summary['Success Test']/lexi_jpt_summary['Total Test']
lexi_jpt_summary['Jump Page CTR'] = lexi_jpt_summary['Jump Page Clicks']/lexi_jpt_summary['Delivered']

lexi_jpt_summary['win'] = lexi_jpt_summary['Success Rate'] != lexi_jpt_summary.groupby(['Hitpath Offer ID'])['Success Rate'].transform('min')
lexi_jpt_summary.loc[(lexi_jpt_summary['Success Rate']==0.5) & (lexi_jpt_summary['Jump Page CTR'] == lexi_jpt_summary.groupby(['Hitpath Offer ID'])['Jump Page CTR'].transform('max')),'win'] = True
lexi_jpt_summary.drop(columns = ['Jump Page Clicks','Delivered'],inplace = True)
lexi_jpt_summary = lexi_jpt_summary.merge(oms[['Hitpath Offer ID','Scheduling Name']],on = 'Hitpath Offer ID',how = 'left')
lexi_jpt_summary = lexi_jpt_summary[['Hitpath Offer ID','Scheduling Name','Jump Page Version','Jump Page Offer','Total Test','Success Rate','Jump Page CTR','win']]
# we already have 2 test on the jump page version but all test is failed. We don't need to test it anymore 
underperforming_jumppage= lexi_jpt_summary.loc[(lexi_jpt_summary['Total Test'] == 2)  & (lexi_jpt_summary['Success Rate'] ==0) & ((lexi_jpt_summary['Jump Page Version']!= 0 )) ,'Jump Page Offer'].unique().tolist()
passed_jumppage = lexi_jpt_summary.loc[(lexi_jpt_summary['Total Test'] == 2)  & (lexi_jpt_summary['Success Rate'] ==1) & ((lexi_jpt_summary['Jump Page Version']!= 0 )) ,'Jump Page Offer'].unique().tolist()

In [5]:
lexi_sms_JPT_simplified = lexi_sms_JPT_simplified[['Hitpath Offer ID','Scheduling Name','Date','SC_DP&Pub','Jump Page Offer','Revenue','Delivered','eCPM','Jump Page Click Rate','JPCTR50','Success Test']].sort_values(by = ['Hitpath Offer ID','SC_DP&Pub'],ascending = False)
lexi_sms_JPT_simplified

,Hitpath Offer ID,Scheduling Name,Date,SC_DP&Pub,Jump Page Offer,Revenue,Delivered,eCPM,Jump Page Click Rate,JPCTR50,Success Test
66,9088,9088-Resources-Smooth Marketing-EXCLUSIVE - CP...,2023-11-19,UAA_FSM.YS_461242,9088,43.50,7923.0,5.490345,0.295964,0.461485,True
67,9088,9088-Resources-Smooth Marketing-EXCLUSIVE - CP...,2023-11-19,UAA_FSM.YS_461242,9088v1,21.00,2000.0,10.500000,0.144186,0.342148,False
64,9088,9088-Resources-Smooth Marketing-EXCLUSIVE - CP...,2023-11-17,MBC_PA.PS_461247,9088,31.50,2498.0,12.610088,0.516014,0.850144,False
65,9088,9088-Resources-Smooth Marketing-EXCLUSIVE - CP...,2023-11-17,MBC_PA.PS_461247,9088v1,34.50,958.0,36.012526,0.631250,1.348650,True
62,9088,9088-Resources-Smooth Marketing-EXCLUSIVE - CP...,2023-11-17,HZB_FSM.YS_461242,9088,45.00,2694.0,16.703786,0.604938,1.025444,True
...,...,...,...,...,...,...,...,...,...,...,...
5,11600,11600-Resources-What If Media-Unemployments Be...,2024-04-02,FLC_PN.SWP_461500,11600v1,300.85,18573.0,16.198245,0.659138,1.087301,False
2,11468,11468-Resources-Smooth Marketing-R - CPL - Rec...,2023-11-13,HZB_TLG.PL_461768,11468,61.50,4887.0,12.584408,0.513514,0.846556,True
3,11468,11468-Resources-Smooth Marketing-R - CPL - Rec...,2023-11-13,HZB_TLG.PL_461768,11468v1,36.00,3897.0,9.237875,0.286713,0.505821,False
0,11468,11468-Resources-Smooth Marketing-R - CPL - Rec...,2024-02-29,HZB_PA.PS_461247,11468,21.00,3138.0,6.692161,0.000000,0.100336,False


In [6]:
# dataframe that is awaiting to be test 
offer_performance_awaiting_test = offer_performance_awaiting_test.loc[offer_performance_awaiting_test['Jump Page Offer'].isin(underperforming_jumppage + passed_jumppage)== False,]
offer_performance_awaiting_test.rename(columns = {'Test count':'# drops awaiting test'},inplace = True)
offer_performance_awaiting_test = offer_performance_awaiting_test[['Jump Page Offer','Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status','# drops awaiting test']]
offer_performance_awaiting_test

,Jump Page Offer,Hitpath Offer ID,Scheduling Name,Advertiser,RX Rep,Status,# drops awaiting test
13,13070v1,13070,13070-Home Warranty-Globalwide Media-Choice Ho...,Globalwide Media,Sarah Bowe,Live: Budgeted Offer,3.0
2,6444v1,6444,6444-Credit Score-Globalwide Media-ClickFreeSc...,Globalwide Media,Sarah Bowe,Live,1.0
10,12945v1,12945,12945-Resources-Three Hyphens LLC-LavishGreen ...,Three Hyphens LLC,Marley Smith,Live,2.0
4,12588v1,12588,12588-Loan-EPCVIP-Money Loan USA,EPCVIP,Kelly baker,Live,1.0
6,12088v1,12088,12088-Education-Banner Edge-Online EDU Degrees...,Banner Edge,Rhiannon Selander,Live,2.0
9,12088v2,12088,12088-Education-Banner Edge-Online EDU Degrees...,Banner Edge,Rhiannon Selander,Live,3.0
0,12305v1,12305,12305-Loan-The Lead Group-Market Loans - Perso...,The Lead Group,Jennifer Sporleder,Live,1.0
8,11544v1,11544,11544-Education-Path56-The College Review - Pa...,Path56,Jennifer Sporleder,Live,3.0
12,12699v1,12699,12699-Home Warranty-Massive (c3 Data)-First Am...,Massive (c3 Data),Jennifer Sporleder,Live,3.0
15,13476v1,13476,13476-Credit Score-Media Fit-Dovly - Boost You...,Media Fit,Haley Bush,Live,3.0


In [7]:
# Dataframe that is requesting more jump page  
request_jp = request_jp[['Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status','Revenue','Text CTR','eCPM','Jump Page CTR','Jump Page Click Rate','JPCTR50']]
request_jp

,Hitpath Offer ID,Scheduling Name,Advertiser,RX Rep,Status,Revenue,Text CTR,eCPM,Jump Page CTR,Jump Page Click Rate,JPCTR50
11,12145,12145-Survey-What If Media-The American Survey...,What If Media,Hartley Goode,Live,102.00000,0.206488,6.363069,0.006800,0.032931,0.241126
18,12587,12587-Loan-EPCVIP-UnitedLoans,EPCVIP,Kelly baker,Live,682.70000,0.137742,13.717926,0.019631,0.142524,0.528947
0,10086,10086-Credit Card-Brite Box-Now Platinum,Brite Box,Marley Smith,Live,588.00000,0.113475,7.944228,0.016713,0.147279,0.316112
30,13059,13059-Credit Score-Cx3-EXCLUSIVE - 2024FreeScores,Cx3,Mary Beth Howe,Live,1596.00000,0.108755,10.150444,0.029237,0.268830,0.418344
47,13387,13387-Home Sale-Cx3-Smart Timeshare Owner [D] ...,Cx3,Mary Beth Howe,Live,279.00000,0.088419,4.672115,0.006648,0.075189,0.180081
20,12610,12610-Loan-AvenueLink-City-Loans - AvenueLink ...,AvenueLink,Jennifer Sporleder,Live,2330.18000,0.115490,22.193036,0.036239,0.313789,0.863945
9,12110,12110-Resources-Addicted Affiliate-Grant Finde...,Addicted Affiliate,Surya Hemanth,Live,0.00000,0.098171,0.000000,0.029268,0.298137,0.053631
34,13089,13089-Home Insurance-B2Direct-HomeSafeGuarded ...,B2Direct,Hartley Goode,Live,156.00000,0.070973,4.426913,0.003235,0.045582,0.165016
22,12640,12640-Resources-Pushnami-Found Cash,Pushnami,Rhiannon Selander,Live,2010.00000,0.101617,10.567546,0.033091,0.325642,0.440395
6,11890,11890-Credit Card-3 West Media-Milestone Maste...,3 West Media,Sarah Bowe,Live,3276.00000,0.091676,9.782112,0.028486,0.310729,0.403733


In [8]:
# dataframe that is already scheduled JP Test 

all_jump_page_schedule = all_jump_page_schedule.sort_values(by = 'Last Date',ascending = False)
all_jump_page_schedule = all_jump_page_schedule[['Jump Page Offer','Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status','# Drops','Last Date','Revenue','Jump Page Clicks','Delivered','Jump Page CTR','eCPM']]
all_jump_page_schedule

,Jump Page Offer,Hitpath Offer ID,Scheduling Name,Advertiser,RX Rep,Status,# Drops,Last Date,Revenue,Jump Page Clicks,Delivered,Jump Page CTR,eCPM
6,12342v1,12342,12342-Loan-The Lead Group-PickALender.com Pers...,The Lead Group,Jennifer Sporleder,Live,3,2024-07-06,37.510,90.0,3603.0,0.024979,10.410769
9,12945v1,12945,12945-Resources-Three Hyphens LLC-LavishGreen ...,Three Hyphens LLC,Marley Smith,Live,1,2024-07-06,15.000,56.0,867.0,0.064591,17.301038
10,12978v1,12978,12978-Loan-W4/DMS-[a] - Exclusive - Fast Funds...,W4/DMS,Marley Smith,Live,2,2024-07-05,40.000,157.0,2021.0,0.077684,19.792182
11,13113v1,13113,13113-Auto Insurance-Kissterra-Auto Insurance ...,Kissterra,Eamon McMahon,Live,3,2024-07-04,237.726,50.0,11644.0,0.004294,20.416180
14,6322v1,6322,6322-Credit Card-Brite Box-Vast Platinum Card,Brite Box,Marley Smith,Live,5,2024-07-03,133.000,334.0,9049.0,0.036910,14.697757
2,11646v1,11646,11646-Credit Card-B2Direct-Choice Gold Card v2...,B2Direct,Hartley Goode,Live,3,2024-07-03,43.000,104.0,2509.0,0.041451,17.138302
12,13194v1,13194,13194-Loan-AvenueLink-Funds-Now.com - AvenueLi...,AvenueLink,Jennifer Sporleder,Live,4,2024-07-02,258.450,406.0,6437.0,0.063073,40.150691
13,13221v1,13221,13221-Credit Card-B2Direct-PlatinumCardApprova...,B2Direct,Hartley Goode,Live,3,2024-07-02,51.000,172.0,4882.0,0.035231,10.446538
4,12089v2,12089,12089-Education-Banner Edge-Degree Snap - SMS ...,Banner Edge,Rhiannon Selander,Live,2,2024-06-26,0.000,11.0,5829.0,0.001887,0.000000
3,12088v1,12088,12088-Education-Banner Edge-Online EDU Degrees...,Banner Edge,Rhiannon Selander,Live,1,2024-06-25,35.000,7.0,1500.0,0.004667,23.333333


In [9]:
# create a workbook that I can save all the following tap into a excel file 
#all_jump_page_schedule
#request_jp
#offer_performance_awaiting_test
#lexi_jpt_summary
#lexi_sms_JPT_simplified

from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule, FormulaRule
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
# create a workbook that I can save all the following tap into a excel file
# Define a function to apply conditional formatting

file_path = 'jump_page_feedback_reports.xlsx'  

# Create an Excel writer object and specify the path of the Excel file
with pd.ExcelWriter(file_path, engine= 'openpyxl') as writer:
    offer_performance_awaiting_test.to_excel(writer, sheet_name='Awaiting Test - Jump Page',index=False)
    request_jp.to_excel(writer, sheet_name='Need Request - Jump Page', index=False)
    all_jump_page_schedule.to_excel(writer, sheet_name='All Jump Page Test Schedule', index=False)
    lexi_jpt_summary.to_excel(writer, sheet_name='Jump Page Testing Summary',index=False)
    lexi_sms_JPT_simplified.to_excel(writer, sheet_name='Jump Page Test Result Details',index=False)
# Open the Excel file with openpyxl
workbook = openpyxl.load_workbook(file_path)

def merge_cells_if_same(sheet):
    for col in sheet.iter_cols(min_col=1, max_col=3):
        cell_value = None
        merge_start = None
        for cell in col:
            if cell.value != cell_value:
                if merge_start is not None and merge_start != cell.row - 1:
                    sheet.merge_cells(start_row=merge_start, start_column=cell.column, end_row=cell.row - 1, end_column=cell.column)
                    merged_cell = sheet.cell(row=merge_start, column=cell.column)
                    merged_cell.alignment = Alignment(vertical='center')
                cell_value = cell.value
                merge_start = cell.row
        if merge_start is not None and merge_start != sheet.max_row:
            sheet.merge_cells(start_row=merge_start, start_column=col[0].column, end_row=sheet.max_row, end_column=col[0].column)
            merged_cell = sheet.cell(row=merge_start, column=col[0].column)
            merged_cell.alignment = Alignment(vertical='center')

# Apply merging function to all sheets

sh = workbook['All Jump Page Test Schedule']
for col in sh.columns:
    col_letter = col[0].column_letter
    sh.column_dimensions[col_letter].width = 20
for cell in sh['L']:
    cell.number_format = '0.00%'  # Percentage with two decimal places
for cell in sh['I']:
    cell.number_format = '$#,##0.00'
for cell in sh['M']:
    cell.number_format = '$#,##0.00'


sh = workbook['Need Request - Jump Page']
for col in sh.columns:
    col_letter = col[0].column_letter
    sh.column_dimensions[col_letter].width = 20
for i in ["G",'I','J','K']:
    for cell in sh[i]:
        cell.number_format = '0.00%'  # Percentage with two decimal places
for i in ["F",'H']:
    for cell in sh[i]:
        cell.number_format = '$#,##0.00'

sh = workbook['Awaiting Test - Jump Page']
for col in sh.columns:
    col_letter = col[0].column_letter
    sh.column_dimensions[col_letter].width = 20

sheet1 = workbook['Jump Page Testing Summary']
merge_cells_if_same(sheet1)
for col in sheet1.columns:
    col_letter = col[0].column_letter
    sheet1.column_dimensions[col_letter].width = 20
green_fill = PatternFill(start_color="A9DFBF", end_color="A9DFBF", fill_type="solid")
sheet1.conditional_formatting.add('H1:H10000',
                                 CellIsRule(operator='equal', formula=['TRUE'], stopIfTrue=True, fill=green_fill))
for i in ["F",'G']:
    for cell in sheet1[i]:
        cell.number_format = '0.00%' 


sheet = workbook['Jump Page Test Result Details']
merge_cells_if_same(sheet)
for col in sheet.columns:
    col_letter = col[0].column_letter
    sheet.column_dimensions[col_letter].width = 20
sheet.conditional_formatting.add('K1:K10000',
                                 CellIsRule(operator='equal', formula=['TRUE'], stopIfTrue=True, fill=green_fill))
for i in ['I','J']:
    for cell in sheet[i]:
        cell.number_format = '0.00%'  # Percentage with two decimal places
for i in ["F",'H']:
    for cell in sheet[i]:
        cell.number_format = '$#,##0.00'


# Save the modified Excel file
workbook.save(file_path)
""" 
Loading the Excel File: The script starts by loading the Excel file and reading all sheets into a dictionary of DataFrames.
Saving the DataFrames: It then saves these DataFrames back to the Excel file to ensure the format is compatible with openpyxl.
Merging Cells: The merge_cells_if_same function iterates over each column in the sheet, checking for consecutive cells with the same value and merging them.
Applying the Function: The merging function is applied to all sheets in the workbook.
Saving the Workbook: Finally, the modified workbook is saved back to the specified file path.
Replace 'path_to_your_excel_file.xlsx' with the actual path to your Excel file. This script will merge cells vertically within each column where the cells have the same value.
"""



" \nLoading the Excel File: The script starts by loading the Excel file and reading all sheets into a dictionary of DataFrames.\nSaving the DataFrames: It then saves these DataFrames back to the Excel file to ensure the format is compatible with openpyxl.\nMerging Cells: The merge_cells_if_same function iterates over each column in the sheet, checking for consecutive cells with the same value and merging them.\nApplying the Function: The merging function is applied to all sheets in the workbook.\nSaving the Workbook: Finally, the modified workbook is saved back to the specified file path.\nReplace 'path_to_your_excel_file.xlsx' with the actual path to your Excel file. This script will merge cells vertically within each column where the cells have the same value.\n"

In [10]:
import send_email 

good_test = lexi_jpt_summary[lexi_jpt_summary['Total Test'] >=2]
bad_test = good_test[(good_test['win'] ==False ) & (good_test['Jump Page Version']!=0)]
good_test = good_test[(good_test['win'] ==True ) & (good_test['Jump Page Version']!=0)]
good_test_html = good_test.to_html(index = False)
body = """
<html>
<head></head>
<body>
<p>Hello,</p>
<p>We passed the following Jump Page versions and please test:</p>
""" + good_test_html + """
<p>Regards,<br>Lili Guo</p>
</body>
</html>
"""

toaddr = ['sms@rxmg.com','offernotice@rxmg.com']
#toaddr = ['lili@rxmg.com']
for i in toaddr: 
    send_email.send_email([file_path],'Jump Page Feedback Report', body,i,'html')


# 3. Create Jump Page Management Database 

In [11]:
offerlist_jumppage

869     12305v1
918      6322v1
919      6322v1
920      6322v1
921      6322v1
         ...   
2935    13476v1
2936    13476v1
2937    13476v1
2938    13476v1
2939    13476v1
Name: 0, Length: 141, dtype: object

In [12]:
jpdatabase = pd.DataFrame(offerlist_jumppage.unique(),columns = ['Jump Page Offer'])
sms_ss_sc = infrastructure.get_gc().open_by_url('https://docs.google.com/spreadsheets/d/1n8qaY4bxSKVDGc0bds1o33toLa7g9G9-T-nKDwlECVs/edit?gid=2010995119#gid=2010995119')
jp_vesions_link  = sms_ss_sc.worksheet('title','JP versions').get_as_df()
jp_link = sms_ss_sc.worksheet('title','Jump Management').get_as_df()
jp_link = jp_link[jp_link['SC']!='CSS']
jp_link_each = jp_link.groupby('sid')['new link'].first().reset_index()
jp_link_each.rename(columns = {'sid':'Jump Page Offer','new link':'Jump Page Link'},inplace = True)
jp_offer = jp_link_each[['Jump Page Offer','Jump Page Link']]
jpdatabase = pd.concat([jpdatabase,jp_link_each],axis = 0).reset_index(drop = True)
jpdatabase['Hitpath Offer ID'] = jpdatabase['Jump Page Offer'].str.split("v",expand = True)[0]
jpdatabase.loc[jpdatabase['Hitpath Offer ID'].isna(), 'Hitpath Offer ID'] = jpdatabase['Jump Page Offer']
jpdatabase['Hitpath Offer ID'] = jpdatabase['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
jpdatabase = jpdatabase.merge(oms[['Hitpath Offer ID','Scheduling Name','Advertiser','RX Rep','Status']],on = 'Hitpath Offer ID',how = 'left')
jpdatabase['Test Result'] = ''
jpdatabase.loc[jpdatabase['Jump Page Offer'].str.contains('v',na = False) == False, 'Test Result'] = 'Original Version'
jpdatabase.loc[jpdatabase['Status'].isin(['Retest','In Testing']), 'Test Result'] = 'Pipeline Testing'
jpdatabase.loc[jpdatabase['Jump Page Offer'].isin(good_test['Jump Page Offer'].unique().tolist()), 'Test Result'] = 'Passed Test'
jpdatabase.loc[jpdatabase['Jump Page Offer'].isin(bad_test['Jump Page Offer'].unique().tolist()) , 'Test Result'] = 'Failed Test'
jpdatabase.loc[jpdatabase['Jump Page Offer'].isin(offer_performance_awaiting_test['Jump Page Offer'].unique().tolist()), 'Test Result'] = 'In Testing'
jpdatabase.sort_values(by = ['Advertiser','RX Rep','Hitpath Offer ID','Jump Page Offer'],ascending = True,inplace = True)
jpdatabase = jpdatabase[['Hitpath Offer ID','Jump Page Offer','Jump Page Link','Scheduling Name','Advertiser','RX Rep','Status','Test Result']]

# get one link for each jump page offer
jp_vesions_link_each = jp_vesions_link.groupby('sidversion')['new link'].first().reset_index()

jpdatabase = jpdatabase.merge(jp_vesions_link_each,left_on = 'Jump Page Offer',right_on = 'sidversion',how = 'left')
jpdatabase.loc[jpdatabase['Jump Page Link'].isna(), 'Jump Page Link'] = jpdatabase['new link']
jpdatabase = jpdatabase[['Hitpath Offer ID','Jump Page Offer','Jump Page Link','Scheduling Name','Advertiser','RX Rep','Status','Test Result']]
jpdatabase = jpdatabase.loc[jpdatabase['Scheduling Name'].isna() == False]
jpdatabase = jpdatabase.sort_values(by = ['Advertiser','RX Rep','Hitpath Offer ID','Jump Page Offer'],ascending = True)

In [13]:
jpdatabase[jpdatabase['RX Rep'].isna()]

,Hitpath Offer ID,Jump Page Offer,Jump Page Link,Scheduling Name,Advertiser,RX Rep,Status,Test Result
42,6731,6731,https://cf.horizonbenefits.org/?sid=6731&pub={...,6731-Auto Loan-Amobee-ACE Auto Loans CPL - Amo...,Amobee,NaN,Paused: All Else,Original Version
125,9980,9980,https://cf.horizonbenefits.org/?sid=9980&pub={...,9980-Gift Card-Conceptual Media-RXMG $500 Gas ...,Conceptual Media,NaN,Paused: Internal,Original Version
190,12067,12067,https://cf.freedomlender.co/?sid=12067&pub={{d...,12067-Internet-Keono-Internet-Keono-AT&T Inter...,Keono,NaN,Paused: All Else,Original Version
293,9643,9643,https://cf.horizonbenefits.org/?sid=9643&pub={...,9643-Credit Card-Renown Holdings-Accepted Lead...,Renown Holdings,NaN,Paused: Failed Testing,Original Version
400,11915,11915,https://cf.horizonbenefits.org/?sid=11915&pub=...,11915-Resources-eFlow-GI Discounts - Military ...,eFlow,NaN,Paused: Failed Testing,Original Version
401,12011,12011,https://cf.freedomlender.co/?sid=12011&pub={{d...,12011-Legal-eFlow-GI Discounts - Mass Tort (SMS),eFlow,NaN,Paused: Failed Testing,Original Version


In [14]:
import smartsheet
import os 
def place_in_smartsheet(df, key, sheet_id):
    """
    Writes a dataframe to a given smartsheet

    :param df: a DataFrame whose column names match with columns in the given smartsheet
    :param key: a string of the smartsheet API token
    :param sheet_id: an int of the smartsheet sheet ID

    :return: the result of placing into smartsheet. Use help() if you need to know more. 
             E.x. result = place_in_smartsheet(df, key, sheet_id) --> help(result)
    """
    print('Adding Data to Smartsheet...')

    smart = smartsheet.Smartsheet(key)
    sheet = smart.Sheets.get_sheet(sheet_id)
    am_contact_list = { 'Eamon McMahon':'e.mcmahon@rxmg.com' , 'Haley Bush':'h.bush@rxmg.com', 'Hartley Goode':'h.goode@rxmg.com', 'Jennifer Chatellier':'jennifer@rxmg.com', 'Jennifer Sporleder':'j.sporleder@rxmg.com', 
                   'Kelly baker':'kelly@rxmg.com', 'Marley Smith':'m.smith@rxmg.com', 'Mary Beth Howe':'m.howe@rxmg.com', 'Rhiannon Selander':'r.selander@rxmg.com', 'Sarah Bowe':'s.bowe@rxmg.com', 'Surya Hemanth':'s.hemanth@rxmg.com'}
    smartsheet_client = smartsheet.Smartsheet(key)
    download_path = filepath.smartsheet_folder
    #smartsheet_client_lili_2 = smartsheet.Smartsheet('7fQH9Go8sf6CruwY3g4HHzKHe6wFMzNbgyAYy') #  at token is fully used, we need extra token to load content test csv 
    jp_sheet=smartsheet_client.Sheets.get_sheet_as_csv(sheet_id,download_path,'SMS Jump Page Management.csv')
    jp_sheet_df = pd.read_csv(os.path.join(download_path,'SMS Jump Page Management.csv'))
    
    df= df.merge(jp_sheet_df[['Jump Page Offer','Promote?']],how = 'left',on = 'Jump Page Offer')
    df['Promote?'] = df['Promote?'].fillna(False)
    df = df.fillna('')
    # translate column names to column id
    column_map = {}
    for column in sheet.columns:
        column_map[column.title] = (column.id, column.type)

    data_dict = df.to_dict('index')
    # Delete all existing rows in the sheet
    existing_row_ids = [row.id for row in sheet.rows]
    if existing_row_ids:
        smart.Sheets.delete_rows(sheet_id, existing_row_ids)

    rows_list = []
    for _, i in data_dict.items():
        row = smart.models.Row()
        row.to_bottom = True

        for k, v in i.items():
            cell = smart.models.Cell()
            cell.column_id = column_map[k][0]
            cell.column_type = str(column_map[k][1])
            if k == 'RX Rep' and v in am_contact_list:
                cell.object_value = {
                    "objectType": "CONTACT",
                    "email": am_contact_list[v],  
                    "name": v
                }
            else:
                cell.object_value = v
            row.cells.append(cell)
        
        rows_list.append(row)
        
    result = smart.Sheets.add_rows(sheet_id, rows_list)

    print('Data added to Smartsheet')

    return result

In [15]:
# Smartsheet API token
#key = 'gMX4xu48SUIH7rMEj4iaj7rIsQJ2ERbmgiYMJ'
key = '7fQH9Go8sf6CruwY3g4HHzKHe6wFMzNbgyAYy' # backup key
sheet_id = 7742257205563268
result = place_in_smartsheet(jpdatabase, key, sheet_id)

Adding Data to Smartsheet...
Data added to Smartsheet
